In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

from flask import jsonify
from flask import request
from flask_api import FlaskAPI
from sqlalchemy import create_engine
import json

In [15]:
#Data Scrapping
base_URL = "https://www.freshfields.com"
URL = "https://www.freshfields.com/en-gb/contacts/find-a-lawyer/?Name=&t=&Service=97&Role=&Location=&Office=&Industry=&Page="
columns = ["URL","Name","Role","Fax","Mobile","Telephone","Address","Country","Email"]

data = []

def get_base_content(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id = "main-content")
    return results

def details_links():
    detail_links = []
    result = get_base_content(URL)
    navs = result.find("div",class_="row fal-flex").find_previous("div").find_all(alt="pageCount")
    for nav in navs:
        results = get_base_content(base_URL+nav['href'])
        contents = results.find("div", class_="row fal-flex")
        for i in contents.find_all('a'):
            detail_links.append(base_URL+i['href'])
    return detail_links

def user_details(URL):
    _user = []
    results = get_base_content(URL)
    name_role = results.find("a",class_="hero-link").find_previous("h1")
    _user.append(URL) #url
    _user.append(name_role.text.strip().split(", ")[0]) #name
    _user.append(name_role.text.strip().split(", ")[1]) #role
    
    adr = results.find("address")
    ads = []
    
    for i in adr.text.strip().split("\n"):
        if(len(i)>1):
            ads.append(i.strip())
            
    spl = " ".join(ads).split("F +")
    _user.append("+"+spl.pop().strip()) #fax
    
    spl = spl[0].split("M +")
    if(len(spl) == 2):
        _user.append("+"+spl.pop().strip()) #phone
    else:
        _user.append("null")
        
    spl = spl[0].split("T +")
    _user.append("+"+spl.pop().strip()) #telephone
    
    _user.append(spl.pop().strip()) #address
    
    _user.append(results.find("address").find_previous('h3').text) #country
    _user.append(results.find("address").find_next('a').text) #mail
    
    return dict(zip(columns,_user))

def get_data():
    final_list = []
    ulrs = details_links()
    for url in ulrs:
        final_list.append(user_details(url))
    return final_list

In [7]:
data = get_data() #execute to get the data

In [13]:
#API
app = FlaskAPI(__name__)

e = create_engine('sqlite:///requests_flask_sqlite.db')


@app.route('/freshfields_create_table', methods=['POST']) #need to pass the table name Eg: /create_table?tbl=tbl_name
def freshfields_create_table():
    try:
        tbl = request.args.get('tbl')
        conn = e.connect()
        conn.execute('''CREATE TABLE IF NOT EXISTS {} (
                        SNO INTEGER PRIMARY KEY AUTOINCREMENT,
                        URL TEXT,NAME TEXT,ROLE TEXT,ADDRESS TEXT,
                        COUNTRY TEXT,MOBILE TEXT,TELEPHONE TEXT,
                        FAX TEXT,EMAIL TEXT,DATE_TIME TIMESTAMP DEFAULT (datetime('now','localtime')));'''.format(tbl))
        
        text = {"Message " : "Table created successfully"}
        return text
    except Exception as error:
        text = {"Message " : "Table creation failed : {}".format(error)}
        return text
    finally:
        conn.close()

@app.route('/freshfields_view_all', methods=['GET']) #Eg: /tbl_view_all?tbl=tbl_name
def freshfields_view_all():
    try:
        tbl = request.args.get('tbl')
        conn=e.connect()
        result = conn.execute("select * from {}".format(tbl))
        return json.dumps([dict(r) for r in result])
    except Exception as error:
        text = {"Message " : "Table not found : {}".format(error)}
        return text
    finally:
        conn.close()

@app.route('/freshfields_view_one', methods=['GET']) #Eg: /tbl_view_one?tbl=tbl_name&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/f/french-daniel/
def freshfields_view_one():
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        url = all_args.get('url')
        result = conn.execute("select * from {} where URL = '{}';".format(tbl,url))  
        return json.dumps([dict(r) for r in result])
    
    except Exception as error:
        text = {"Message " : "Record not found : {}".format(error)}
        return text
    finally:
        conn.close()
    
@app.route('/freshfields_insert_one', methods=['POST']) #Eg: /tbl_insert_one?tbl=tbl_name&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/f/french-daniel/
def freshfields_insert_one():    
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        url = all_args.get('url')
        result = user_details(url)
        conn.execute('''INSERT INTO {} (URL,NAME,ROLE,ADDRESS,COUNTRY,MOBILE,TELEPHONE,FAX,EMAIL)
                    VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')'''
                        .format(tbl,result.get("URL"),result.get("Name"),result.get("Role"),
                          result.get("Address"),result.get("Country"),result.get("Mobile"),
                            result.get("Telephone"),result.get("Fax"),result.get("Email")))
        
        text = {"Message " : "Record inserted successfully"}
        return text
    
    except Exception as error:
        text = {"Message " : "Insertion failed : {}".format(error)}
        return text
    finally:
        conn.close()
    
@app.route('/freshfields_insert', methods=['POST']) #Eg: /freshfields_insert?tbl=tbl_name
def freshfields_insert():    
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        for result in data:
            conn.execute('''INSERT INTO {} (URL,NAME,ROLE,ADDRESS,COUNTRY,MOBILE,TELEPHONE,FAX,EMAIL)
                        VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}');'''
                             .format(tbl,result.get("URL"),result.get("Name"),result.get("Role"),
                                 result.get("Address"),result.get("Country"),result.get("Mobile"),
                                     result.get("Telephone"),result.get("Fax"),result.get("Email")))
            
        text = {"Message " : "Records inserted successfully"}
        return text
    
    except Exception as error:
        text = {"Message " : "Insertion failed : {}".format(error)}
        return text
    finally:
        conn.close()
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Nov/2021 12:07:29] "GET /freshfields_view_all?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 12:07:31] "POST /freshfields_create_table?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 12:07:36] "POST /freshfields_insert?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2021 12:07:38] "GET /freshfields_view_all?tbl=freshfields HTTP/1.1" 200 -


In [16]:
user_details('https://www.freshfields.com/en-gb/contacts/find-a-lawyer/a/almroth-david/')

{'URL': 'https://www.freshfields.com/en-gb/contacts/find-a-lawyer/a/almroth-david/',
 'Name': 'David Almroth',
 'Role': 'Partner',
 'Fax': '+1 646 521 5670',
 'Mobile': 'null',
 'Telephone': '+1 212 277 4070',
 'Address': '601 Lexington Avenue, 31st Floor New York, NY 10022',
 'Country': 'New York',
 'Email': 'david.almroth@freshfields.com'}